# Perform the Classification

## Loading the vector store and metadata

In [302]:
import class_utils as cu
import pandas as pd
import re
import prompts
from class_utils import query_document
import openai
from dotenv import load_dotenv
import os


In [303]:
MODEL = "openai/chatgpt-4o-latest"
TEMPERATURE = 0.0

In [304]:
vector_store = cu.load_vector_store()

In [305]:
names_df = pd.read_excel("../data/julien_gt_revised.xlsx")

In [306]:
names_df = names_df[names_df["First link"].notna() & (names_df["First link"] != "")]

In [307]:
len(names_df)

163

In [308]:
groundtruth_df = names_df[names_df["source"].str.contains("groundtruth", case=False, na=False)]
remaining_df = names_df[~names_df["source"].str.contains("groundtruth", case=False, na=False)]

In [309]:
len(groundtruth_df)

17

In [310]:
import os

summaries_path = '../data/summaries/'

# List all subfolders (these are the titled folders)
folder_names = [d for d in os.listdir(summaries_path) if os.path.isdir(os.path.join(summaries_path, d))]

# The columns we care about (questions 2-6)
question_cols = [f'question_{i}' for i in range(2, 7)]

rows = []

for folder in folder_names:
    folder_dict = {'title': folder}
    folder_path = os.path.join(summaries_path, folder)
    for q in range(2, 7):
        txt_path = os.path.join(folder_path, f'question_{q}_summary.txt')
        if os.path.exists(txt_path):
            with open(txt_path, 'r', encoding='utf-8') as f:
                content = f.read()
            folder_dict[f'question_{q}'] = content
        else:
            folder_dict[f'question_{q}'] = None
    rows.append(folder_dict)

# No index - just let title be a column alongside the other question columns
summaries_df = pd.DataFrame(rows)

In [311]:
summaries_df

,title,question_2,question_3,question_4,question_5,question_6
0,2qn97s0DnXYe2qH8,"The document is a Canadian Securities Administrators (CSA) Staff Notice (CSA Staff Notice 81-334) that addresses ESG-related investment fund disclosure, updated on March 7, 2024. Its primary aim is to clarify and enhance the transparency and consistency of disclosure and communications for investment funds concerning environmental, social, and governance (ESG) matters, thereby facilitating informed investment decisions by investors.\n\nKey highlights of the sectoral focus in the document include:\n\n1. **Domestic Financial Sector Policy**: The guidance aims to influence financial actors and markets by improving the regulatory framework around ESG disclosures. This includes best practices for investment fund managers (IFMs) and clarifications on how existing regulations apply to ESG-related funds. The goal is to mitigate greenwashing, where funds misrepresent their ESG commitments.\n\n2. **ESG-Related Fund Categories**: The document identifies distinct types of funds based on their integration of ESG factors, including:\n - Funds explicitly referencing ESG objectives.\n - Funds that do not reference ESG but incorporate ESG strategies to varying degrees.\n This differentiated approach provides investors with clear expectations regarding disclosure standards depending on the fund type.\n\n3. **Risk Disclosure and Continuous Monitoring**: The notice highlights the importance of consistent risk disclosure, allowing investors to understand the implications of ESG strategies on fund performance. This includes monitoring actual investments against stated ESG objectives and methodologies, further reinforcing the accountability of funds in managing ESG factors.\n\n4. **Proxy Voting and Engagement Policies**: There is emphasis on the need for transparent engagement in proxy voting procedures that align with ESG objectives, ensuring that these strategies are effectively integrated into the funds' operational frameworks.\n\n5. **Sales Communications**: The document discusses the standards for sales communications, stressing that any marketing or advertising must accurately reflect the ESG characteristics of the funds. It discourages misleading representations that may distort the fund's actual ESG commitment.\n\nOverall, the notice reflects an enhanced focus on the domestic financial market's response to investor demand for more sustainable and responsible investment options, through stringent disclosure requirements aimed at both protecting investors and promoting genuine ESG integration within the financial sector.","The document is a revised Staff Notice from the Canadian Securities Administrators (CSA) concerning ESG-related investment fund disclosure, published on March 7, 2024. Its primary aim is to enhance clarity and consistency in the disclosure and marketing practices of investment funds that consider environmental, social, and governance (ESG) factors. This update responds to findings from ESG-focused reviews that indicated the necessity for clearer guidelines to mitigate greenwashing—where funds may mislead investors about their commitment to ESG practices.\n\nThe document outlines the guidelines regarding different types of ESG-related funds, emphasizing the need for clarity in fund names, investment objectives, and sales communications to accurately reflect their ESG focus. It categorizes funds based on the role ESG factors play in their investment processes and sets varying expectations for disclosure based on this role. \n\nKey elements include guidance on:\n1. **Types of Funds**: Differentiating among funds that reference ESG objectives, those that utilize ESG strategies to a significant or limited extent, and non-ESG funds.\n2. **Disclosure Practices**: Emphasizing the importance of providing full, true, and plain disclosure concerning ESG strategies, risks, and performance to prevent misleading claims.\n3. **Sales Communications**: Establis

Need to actually save julien's df with unique id! 

In [312]:
# Find titles in summaries_df not present in names_df["name"]
summaries_titles = set(summaries_df['title'])
names_titles = set(names_df["unique_id"])

missing_in_names = summaries_titles - names_titles
missing_in_summaries = names_titles - summaries_titles

print(f"Titles in summaries_df but not in names_df: (total: {len(missing_in_names)})")
for t in sorted(missing_in_names):
    print(f"  {t}")

print(f"\nTitles in names_df but not in summaries_df: (total: {len(missing_in_summaries)})")
for t in sorted(missing_in_summaries):
    print(f"  {t}")


Titles in summaries_df but not in names_df: (total: 9)
  1h7HM9n1rmhba9u6
  J7rkhV101bxtqTVf
  Jz6K6t4H8AIu6Cpl
  UfKZ345Br95V4oD7
  future electricity fund
  jCL0YFCIYNgE76A5
  jzfpTg1Nl1qhewW1
  rfMG0mHWAROiaf94
  sAmXbkITr3dwkIsK

Titles in names_df but not in summaries_df: (total: 4)
  0uhjJKIpooLf4Z96
  Lj5YCPIEl18rPWZi
  uNTlaYHBvxulZQnt
  wbqBzhF1mFuQRVkh


**Note to self: come back and look at this mismatch but excluding and proceeding for now**

In [313]:
# Remove rows from summaries_df where 'title' not in names_df["unique_id"]
summaries_df = summaries_df[summaries_df['title'].isin(names_df['unique_id'])].reset_index(drop=True)

# Optional: Remove rows from names_df where 'unique_id' not in summaries_df['title']
names_df = names_df[names_df['unique_id'].isin(summaries_df['title'])].reset_index(drop=True)


This also shouldn't be necessay but there is some error in my logic when either generating or saving summaries. For now, we will just get rid of the duplicates

______

In [314]:
groundtruth_list = groundtruth_df["unique_id"].tolist()

In [315]:
print(f"The original groundtruth list has {len(groundtruth_list)} names")

The original groundtruth list has 17 names


In [316]:
groundtruth_summaries = summaries_df[summaries_df['title'].apply(lambda x: x in groundtruth_list)]
remaining_summaries = summaries_df[~summaries_df['title'].apply(lambda x: x in groundtruth_list)]
print(f"The groundtruth summaries list has {len(groundtruth_summaries)} names")
print(f"The remaining (non-groundtruth) summaries list has {len(remaining_summaries)} names")

The groundtruth summaries list has 17 names
The remaining (non-groundtruth) summaries list has 142 names


In [317]:
# Extract vector-store entries for the groundtruth set
vector_store_by_clean_name = {}
for original_name, entry in vector_store.items():
    vector_store_by_clean_name.setdefault(original_name, []).append(
        {
            "doc_name": original_name,
            "embeddings": entry["embeddings"],
            "chunks": entry["chunks"],
        }
    )

groundtruth_vectors = {}
missing_groundtruth = []

# Collect vectors for groundtruth list
for normalized_name in groundtruth_list:
    matches = vector_store_by_clean_name.get(normalized_name)
    if matches:
        groundtruth_vectors[normalized_name] = matches
    else:
        missing_groundtruth.append(normalized_name)

# Collect vectors for remaining (non-groundtruth) list
remaining_vectors = {}
for normalized_name in remaining_summaries['title']:
    # Only collect if not in groundtruth
    if normalized_name not in groundtruth_vectors:
        matches = vector_store_by_clean_name.get(normalized_name)
        if matches:
            remaining_vectors[normalized_name] = matches

print(f"Collected vectors for {len(groundtruth_vectors)} groundtruth names out of {len(groundtruth_list)}.")
print(f"Collected vectors for {len(remaining_vectors)} remaining (non-groundtruth) names out of {len(remaining_summaries)}.")

if missing_groundtruth:
    print("\nGroundtruth names missing from vector store:")
    for name in missing_groundtruth:
        print(f"  {name}")


Collected vectors for 17 groundtruth names out of 17.
Collected vectors for 139 remaining (non-groundtruth) names out of 142.


_______

In [318]:
groundtruth_df = groundtruth_df.rename(columns={
    'Criterion 1': 'question_2',
    'Criterion 2': 'question_3',
    'Criterion 3': 'question_5',
    'Criterion 4': 'question_4',
})

### Extracting few shot examples

In [319]:
row_idx = groundtruth_df[groundtruth_df['unique_id'] == 'eRFNvUxTiyJ5mgky'].index
if not row_idx.empty:
    groundtruth_df.at[row_idx[0], 'question_2'] = 'MIT'


In [320]:
row_idx = groundtruth_df[groundtruth_df['unique_id'] == 'lf0vaU6VpW26FhQV'].index
if not row_idx.empty:
    groundtruth_df.at[row_idx[0], 'question_2'] = 'MIT, ADP'


In [321]:
# Map category code prefixes in question_2 to their full category names for clarity

mapping = {
    "MIT": "Domestic real economy decarbonization measures with a financial component",
    "RED": "Domestic real economy decarbonization measures with a financial component",
    "ADP": "Domestic real economy adaptation and compensation measures with a financial component",
    "MIT, ADP": "Domestic real economy decarbonization measures with a financial component; Domestic real economy adaptation and compensation measures with a financial component",
    "FSP": "Domestic financial sector policy", 
    "ICF": "International climate-focused financial",
}

def replace_prefix_with_fullname(text):
    # Only apply if text is a string (to avoid NaNs)
    if not isinstance(text, str):
        return text
    for short, full in mapping.items():
        # Replace only whole-word/exact prefix matches
        text = re.sub(r'\b%s\b' % re.escape(short), full, text)
    return text

groundtruth_df['question_2'] = groundtruth_df['question_2'].apply(replace_prefix_with_fullname)

In [322]:
# Move only the relevant fields in the target row:
target_name = 'natural infrastructure fund'
row_idx = groundtruth_df[groundtruth_df['name'].str.strip().str.lower() == target_name].index

if not row_idx.empty:
    idx = row_idx[0]
    # Copy over the values as per instruction
    groundtruth_df.at[idx, 'question_3'] = groundtruth_df.at[idx, 'question_2']
    groundtruth_df.at[idx, 'question_4'] = groundtruth_df.at[idx, 'question_3']
    groundtruth_df.at[idx, 'question_5'] = groundtruth_df.at[idx, 'question_4']
    # Optionally: blank out question_2, question_3, question_4 (not requested)


In [323]:
groundtruth_list

['cU3N5G1KqrceLGLz',
 '3Oahxp3pihpVDQVt',
 'tnaWzSNVgSLTv7XX',
 'K96A0SZrnego6D4a',
 'G6RmGheTDI0EhzXd',
 'rxkDG0Ai5LbWIkAw',
 'zm6ny9KbxXZrYyEM',
 'cbRMF5EZrlSlsqq1',
 'eRFNvUxTiyJ5mgky',
 'h8BSZsAJj6qzTP6q',
 'lf0vaU6VpW26FhQV',
 'vAu7yXhg2dOCsQzf',
 'QHLNGcTNy1519Q5Y',
 'DAum9XBl9BxXQdTP',
 '2qn97s0DnXYe2qH8',
 'OzXmCrgCYiGc1Nk2',
 'zxGspsL29aeU1L5v']

In [324]:
# Move specified examples into their own storages: few_shot_list, few_shot_summaries, few_shot_vector_store, and few_shot_df

examples_to_extract = [
    'cU3N5G1KqrceLGLz',
    'h8BSZsAJj6qzTP6q',
    'zxGspsL29aeU1L5v'
]

# Prepare new storages
few_shot_list = []
few_shot_summaries = {}
few_shot_vector_store = {}

# --- Remove examples from groundtruth_list and add to few_shot_list ---
few_shot_list = [x for x in groundtruth_list if x in examples_to_extract]
groundtruth_list = [x for x in groundtruth_list if x not in examples_to_extract]

# --- Remove examples from groundtruth_df and create few_shot_df ---
few_shot_df = None
if isinstance(groundtruth_df, type(None)):
    few_shot_df = None  # nothing to do
else:
    # Find key column and filter for few_shot_df and update groundtruth_df
    if "unique_id" in groundtruth_df.columns:
        few_shot_df = groundtruth_df[groundtruth_df["unique_id"].isin(examples_to_extract)].reset_index(drop=True)
        groundtruth_df = groundtruth_df[~groundtruth_df["unique_id"].isin(examples_to_extract)].reset_index(drop=True)
    elif "title" in groundtruth_df.columns:
        few_shot_df = groundtruth_df[groundtruth_df["title"].isin(examples_to_extract)].reset_index(drop=True)
        groundtruth_df = groundtruth_df[~groundtruth_df["title"].isin(examples_to_extract)].reset_index(drop=True)
    elif "normalized_name" in groundtruth_df.columns:
        few_shot_df = groundtruth_df[groundtruth_df["normalized_name"].isin(examples_to_extract)].reset_index(drop=True)
        groundtruth_df = groundtruth_df[~groundtruth_df["normalized_name"].isin(examples_to_extract)].reset_index(drop=True)
    else:
        few_shot_df = None  # If key column is unknown, skip

# --- Remove/move from groundtruth_summaries ---
if hasattr(groundtruth_summaries, "set_index"):
    # Try to index by unique_id first, then 'title', then 'normalized_name'
    if "unique_id" in groundtruth_summaries.columns:
        gts_index = groundtruth_summaries.set_index("unique_id")
    elif "title" in groundtruth_summaries.columns:
        gts_index = groundtruth_summaries.set_index("title")
    elif "normalized_name" in groundtruth_summaries.columns:
        gts_index = groundtruth_summaries.set_index("normalized_name")
    else:
        gts_index = groundtruth_summaries

    for key in examples_to_extract:
        if key in gts_index.index:
            few_shot_summaries[key] = gts_index.loc[key].to_dict()

    # Remove from summaries DataFrame using all known id/title columns
    for colname in ["unique_id", "title", "normalized_name"]:
        if colname in groundtruth_summaries.columns:
            groundtruth_summaries = groundtruth_summaries[~groundtruth_summaries[colname].isin(examples_to_extract)].reset_index(drop=True)

# --- Remove/move from vector_store ---
if 'vector_store' in globals() and isinstance(vector_store, dict):
    for key in examples_to_extract:
        if key in vector_store:
            few_shot_vector_store[key] = vector_store.pop(key)


In [325]:
def get_examples(few_shot_summaries, question_col, groundtruth_df):
    """
    Returns examples as a list of messages.
    For question_6: only user asks for summary, assistant gives it.
    For other questions: user asks for summary, assistant gives summary, then user asks prompt, assistant gives answer.
    """
    import prompts  # assuming prompts.py is available in the current environment

    # Determine the short question prompt name for this question column
    if isinstance(question_col, str) and question_col.startswith("question_"):
        try:
            short_prompt_name = f"short_{question_col}_prompt"
            short_prompt = getattr(prompts, short_prompt_name)
        except AttributeError:
            short_prompt = ""
    else:
        short_prompt = ""

    example_keys = [
        'cU3N5G1KqrceLGLz',
        'h8BSZsAJj6qzTP6q',
        'zxGspsL29aeU1L5v'
    ]
    messages_examples = []
    for doc in example_keys:
        # Get the summary for this document and this question_col from few_shot_summaries
        summary_response = ""
        if doc in few_shot_summaries:
            summary_response = few_shot_summaries[doc].get(question_col, "")

        summary_user_content = "Give me a summary of this document [document originally provided to assistant but excluded for length]"

        # Special case: For question_6, only return summary exchange (no Q&A)
        if question_col == "question_6":
            messages_examples.append([
                {
                    "role": "user",
                    "content": summary_user_content
                },
                {
                    "role": "assistant",
                    "document": doc,
                    "response": summary_response
                }
            ])
            continue  # skip to next doc

        # For other questions, proceed as before with Q&A
        # Ensure groundtruth_df has a "unique_id" column and pull the answer
        if "unique_id" not in groundtruth_df.columns:
            print("groundtruth_df must contain a 'unique_id' column")
            continue
        mask = groundtruth_df["unique_id"] == doc
        if not mask.any():
            print(f"unique_id {doc} not found in groundtruth_df")
            continue
        row = groundtruth_df[mask].iloc[0]
        if question_col in row:
            content = row[question_col]
        else:
            print(f"{question_col} not found for doc {doc}")
            continue  # skip if this column missing

        question_user_content = short_prompt if short_prompt else ""

        messages_examples.append([
            {
                "role": "user",
                "content": summary_user_content
            },
            {
                "role": "assistant",
                "document": doc,
                "response": summary_response
            },
            {
                "role": "user",
                "content": question_user_content
            },
            {
                "role": "assistant",
                "document": doc,
                "response": content
            }
        ])
    return messages_examples

In [326]:
question_2_examples = get_examples(few_shot_summaries, 'question_2', few_shot_df)

In [327]:
question_2_examples

[[{'role': 'user',
   'content': 'Give me a summary of this document [document originally provided to assistant but excluded for length]'},
  {'role': 'assistant',
   'document': 'cU3N5G1KqrceLGLz',
   'response': "The document outlines the Indigenous Leadership Fund, a program under Canada's Low Carbon Economy Fund, which allocates up to $180 million by 2029 to support climate action initiatives led by Indigenous peoples. The fund specifically focuses on projects that enhance renewable energy, energy efficiency, and low-carbon heating, aligning with Canada's emissions reduction targets. \n\nThe sectoral focus of the program emphasizes the participation of First Nations, Inuit, and Métis communities in leading climate-related projects. It employs a distinctions-based approach to deliver funding tailored to the unique rights and circumstances of these groups. Eligible projects must not only utilize renewable or low-carbon technologies but also demonstrate broader environmental, social, 

In [328]:
question_3_examples = get_examples(few_shot_summaries, 'question_3', few_shot_df)
question_4_examples = get_examples(few_shot_summaries, 'question_4', few_shot_df)
question_5_examples = get_examples(few_shot_summaries, 'question_5', few_shot_df)
question_6_examples = get_examples(few_shot_summaries, 'question_6', few_shot_df)

____

## Chunking

In [329]:
# Load credentials and instantiate the OpenRouter client
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("Please set OPENROUTER_API_KEY before running embeddings.")

client = openai.OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

EMBEDDING_MODEL = "text-embedding-3-large"
print("OpenRouter client configured.")


OpenRouter client configured.


In [330]:
def get_chunks(query, unique_id, vector_store, k=20):
    """
    Given a query and a number of chunks k, return the top k chunks from the vector store
    for the specified unique_id.
    """
    # Use the pattern from embed_and_chunk.ipynb: 
    # The vector store is a dict. Retrieve the store for this doc, and use query_document
    # (assume query_document(store, doc_name, query, top_k) returns ranked chunks for query).
    try:
        results = query_document(
            store = vector_store,
            doc_name=unique_id,
            query=query,
            top_k=k,
            client=client
        )
        return results
    except Exception as e:
        print(f"Error retrieving chunks for {unique_id}: {e}")
        return []

In [331]:
query = prompts.short_question_2_prompt
unique_id = 'cU3N5G1KqrceLGLz'

In [332]:
example_chunks = get_chunks(query, unique_id, few_shot_vector_store, k=5)

In [333]:
example_chunks

[{'chunk_index': 3,
  'similarity': 0.3741692304611206,
  'text': 'not-for-profit organizations, etc. The call for proposals can fund a variety of project types and sizes, with a maximum ECCC contribution of $6 million per project. The stream supports First Nation-owned and First Nation-led renewable energy, energy efficiency, or low-carbon heating projects that provide benefits to First Nations. Eligible projects must be past the concept stage and ready for deployment or construction, meaning that project planning and pre-feasibility activities have been completed, other funding has been secured or is close to being secured, and partnership agreements, impact or environmental assessments, permitting requirements, and power purchase agreements have been finalized or are near finalization. Proposals that include non-First Nation partners must provide transparent and comprehensive information on the ownership and governance structure of the project, including details on decision-making p

____

## Prompting the LLM

In [334]:
def full_evaluation(client, unique_id, vector_store, summary_df):
    two_chunks = get_chunks(prompts.short_question_2_prompt, unique_id, vector_store, k=5)
    three_chunks = get_chunks(prompts.short_question_3_prompt, unique_id, vector_store, k=5)
    four_chunks = get_chunks(prompts.short_question_4_prompt, unique_id, vector_store, k=5)
    five_chunks = get_chunks(prompts.short_question_5_prompt, unique_id, vector_store, k=5)
    six_chunks = get_chunks(prompts.short_question_6_prompt, unique_id, vector_store, k=5)

    # Access summary_df by 'title'
    row = summary_df[summary_df['title'] == unique_id]
    if row.empty:
        raise KeyError(f"unique_id '{unique_id}' not found in summary_df 'title' column")
    two_summary = row['question_2'].iloc[0]
    three_summary = row['question_3'].iloc[0]
    four_summary = row['question_4'].iloc[0]
    five_summary = row['question_5'].iloc[0]
    six_summary = row['question_6'].iloc[0]

    two_prompt = prompts.get_question_2_prompt(two_summary, two_chunks, question_2_examples)
    three_prompt = prompts.get_question_3_prompt(three_summary, three_chunks, question_3_examples)
    four_prompt = prompts.get_question_4_prompt(four_summary, four_chunks, question_4_examples)
    five_prompt = prompts.get_question_5_prompt(five_summary, five_chunks, question_5_examples)
    six_prompt = prompts.get_question_6_prompt(six_summary, six_chunks)

    conversation_history = []

    # Question 2
    two_messages = [
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": two_prompt}
    ]
    two_api_response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        messages=two_messages
    )
    two_response = two_api_response.choices[0].message.content
    conversation_history.append([
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": two_prompt},
        {"role": "assistant", "content": two_response}
    ])

    print("Question 2 response:")
    print(two_response)

    # Question 3
    three_messages = [
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": three_prompt}
    ]
    three_api_response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        messages=three_messages
    )
    three_response = three_api_response.choices[0].message.content
    conversation_history.append([
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": three_prompt},
        {"role": "assistant", "content": three_response}
    ])

    print("Question 3 response:")
    print(three_response)

    # Question 4
    four_messages = [
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": four_prompt}
    ]
    four_api_response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        messages=four_messages
    )
    four_response = four_api_response.choices[0].message.content
    conversation_history.append([
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": four_prompt},
        {"role": "assistant", "content": four_response}
    ])

    print("Question 4 response:")
    print(four_response)

    # Question 5
    five_messages = [
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": five_prompt}
    ]
    five_api_response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        messages=five_messages
    )
    five_response = five_api_response.choices[0].message.content
    conversation_history.append([
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": five_prompt},
        {"role": "assistant", "content": five_response}
    ])

    print("Question 5 response:")
    print(five_response)

    # Question 6
    six_messages = [
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": six_prompt}
    ]
    six_api_response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        messages=six_messages
    )
    six_response = six_api_response.choices[0].message.content
    conversation_history.append([
        {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": six_prompt},
        {"role": "assistant", "content": six_response}
    ])

    print("Question 6 response:")
    print(six_response)

    # Put responses into a DataFrame indexed by unique_id
    responses_df = pd.DataFrame(
        {
            "question_2": [two_response],
            "question_3": [three_response],
            "question_4": [four_response],
            "question_5": [five_response],
            "question_6": [six_response]
        },
        index=[unique_id]
    )

    return responses_df, conversation_history

In [335]:
import json

In [336]:
def get_correction(client, responses_df, unique_id, conversation_history):
    """
    Second LLM evaluates the first LLM's responses using the full conversation history.
    Returns a dict with question numbers as keys and feedback strings as values.
    """
    corrections = {}
    
    # Map question numbers to indices in conversation_history
    # conversation_history[0] = question 2, [1] = question 3, [2] = question 4, [3] = question 5, [4] = question 6
    question_to_index = {2: 0, 3: 1, 4: 2, 5: 3, 6: 4}
    
    # Evaluate each question (2-6)
    for question_num in range(2, 7):
        question_col = f'question_{question_num}'
        conv_index = question_to_index[question_num]
        
        # Get the conversation history for this question
        if conv_index >= len(conversation_history):
            print(f"  Question {question_num}: No conversation history found")
            continue
        
        question_conversation = conversation_history[conv_index]
        
        # Get model response
        if question_col not in responses_df.columns:
            continue
        if unique_id not in responses_df.index:
            continue
            
        model_response = responses_df.loc[unique_id, question_col]
        
        if pd.isna(model_response) or not model_response:
            continue
        
        # Build the full context from conversation history
        # The conversation history contains: [system, user (with full prompt), assistant]
        system_msg = question_conversation[0] if len(question_conversation) > 0 else None
        user_msg = question_conversation[1] if len(question_conversation) > 1 else None
        assistant_msg = question_conversation[2] if len(question_conversation) > 2 else None
        
        # Create evaluation prompt with full conversation context
        eval_prompt = f"""You are an expert evaluator reviewing a classification response for a climate finance policy instrument.

You have access to the full conversation that led to this response. Here is the complete context:

SYSTEM PROMPT:
{system_msg.get('content', '') if system_msg else 'N/A'}

USER PROMPT (includes document summary, relevant chunks, examples, and classification criteria):
{user_msg.get('content', '')[:2000] if user_msg else 'N/A'}...

ASSISTANT RESPONSE:
{assistant_msg.get('content', '') if assistant_msg else model_response}

Review this response in the context of the full conversation. Consider:
- Does the response properly address all aspects of the user's prompt?
- Is the classification accurate given the document summary and relevant chunks provided?
- Does it follow the examples and classification criteria shown in the prompt?
- Are there any missing elements or incorrect categorizations?
- Could the response be improved in clarity, completeness, or accuracy?

Pay extra close attention to whether or not the specified output strucutre is followed. That should be penalized heavily if it is not followed. 

If the response is correct, well-reasoned, and complete given the context, respond with:
{{}}

If corrections or improvements are needed, respond with:
{{
    "needs_correction": true,
    "feedback": "Specific, actionable feedback on what needs to be improved and how. Reference the context from the conversation (summary, chunks, examples) to explain what should be corrected."
}}

Format your response as JSON."""
        
        try:
            eval_response = client.chat.completions.create(
                model=MODEL,
                temperature=TEMPERATURE,
                messages=[
                    {"role": "system", "content": "You are an expert evaluator for climate finance classification tasks. Review responses critically using the full conversation context and provide constructive feedback."},
                    {"role": "user", "content": eval_prompt}
                ],
                response_format={"type": "json_object"}
            )
            
            result = json.loads(eval_response.choices[0].message.content)
            
            if result.get("needs_correction", False):
                corrections[question_col] = result.get("feedback", "Correction needed")
                print(f"  Question {question_num}: Correction needed")
            else:
                print(f"  Question {question_num}: ✓ No correction needed")
                
        except Exception as e:
            print(f"  Error evaluating question {question_num}: {e}")
            # If evaluation fails, don't mark as needing correction
            print(f"  Question {question_num}: Skipped due to error")
    
    return corrections
    


In [337]:
def perform_correction(client, correction, responses_df, unique_id, vector_store, summary_df):
    """
    Regenerates model responses based on correction feedback.
    For each question needing correction, uses the original prompt structure but adds the feedback.
    """
    # Get summaries for this document
    row = summary_df[summary_df['title'] == unique_id]
    if row.empty:
        print(f"Warning: No summaries found for {unique_id}")
        return responses_df
    
    row = row.iloc[0]
    
    # Map question numbers to their prompt functions and short prompts
    question_configs = {
        2: {
            'short_prompt': prompts.short_question_2_prompt,
            'get_prompt': prompts.get_question_2_prompt,
            'summary': row['question_2'],
            'examples': question_2_examples
        },
        3: {
            'short_prompt': prompts.short_question_3_prompt,
            'get_prompt': prompts.get_question_3_prompt,
            'summary': row['question_3'],
            'examples': question_3_examples
        },
        4: {
            'short_prompt': prompts.short_question_4_prompt,
            'get_prompt': prompts.get_question_4_prompt,
            'summary': row['question_4'],
            'examples': question_4_examples
        },
        5: {
            'short_prompt': prompts.short_question_5_prompt,
            'get_prompt': prompts.get_question_5_prompt,
            'summary': row['question_5'],
            'examples': question_5_examples
        },
        6: {
            'short_prompt': prompts.short_question_6_prompt,
            'get_prompt': prompts.get_question_6_prompt,
            'summary': row['question_6'],
            'examples': None  # Question 6 doesn't use examples
        }
    }
    
    for question_num in range(2, 7):  # Questions 2-6
        question_col = f'question_{question_num}'
        
        if question_col not in correction:
            continue
        
        if question_num not in question_configs:
            print(f"  Skipping {question_col}: No configuration found")
            continue
        
        print(f"\nCorrecting {question_col}...")
        config = question_configs[question_num]
        feedback = correction[question_col]
        current_response = responses_df.loc[unique_id, question_col]
        
        # Get relevant chunks
        chunks = get_chunks(config['short_prompt'], unique_id, vector_store, k=5)
        
        # Get the original prompt (question 6 has different signature)
        if question_num == 6:
            original_prompt = config['get_prompt'](config['summary'], chunks)
        else:
            original_prompt = config['get_prompt'](config['summary'], chunks, config['examples'])
        
        # Create correction prompt
        correction_prompt = f"""The evaluator reviewed your previous response and provided this feedback:

{feedback}

Your previous response was:
{current_response}

Please revise your answer based on this feedback. Use the same format and structure as before, but incorporate the corrections."""
        
        # Combine original prompt with correction feedback
        full_prompt = f"""{original_prompt}

---

CORRECTION REQUEST:
{correction_prompt}"""
        
        messages = [
            {"role": "system", "content": prompts.CLASSIFICATION_SYSTEM_PROMPT},
            {"role": "user", "content": full_prompt}
        ]
        
        try:
            response = client.chat.completions.create(
                model=MODEL,
                temperature=TEMPERATURE,
                messages=messages
            )
            new_answer = response.choices[0].message.content
            responses_df.loc[unique_id, question_col] = new_answer
            print(f"  ✓ Updated {question_col}")
        except Exception as e:
            print(f"  ✗ Error correcting {question_col}: {e}")
    
    return responses_df

In [338]:
def full_evaluation_with_correction(client, unique_id, vector_store, summary_df):
    """
    Complete evaluation pipeline: generate responses, get second LLM opinion, and correct.
    
    Flow:
    1. First LLM generates initial responses
    2. Second LLM (evaluator) reviews responses and provides feedback
    3. First LLM corrects responses based on feedback
    
    Returns:
        responses_df: DataFrame with corrected responses
        correction: Dict with correction feedback for each question
    """
    print(f"\n{'='*60}")
    print(f"Evaluating and correcting: {unique_id}")
    print(f"{'='*60}")
    
    # Step 1: Generate initial responses
    print("\nStep 1: First LLM generating initial responses...")
    responses_df, conversation_history = full_evaluation(client, unique_id, vector_store, summary_df)
    
    # Step 2: Second LLM evaluates and provides feedback
    print("\nStep 2: Second LLM reviewing responses...")
    correction = get_correction(client, responses_df, unique_id, conversation_history)
    
    # Step 3: Apply corrections if any are needed
    if correction:
        print(f"\nStep 3: First LLM applying corrections for {len(correction)} question(s)...")
        responses_df = perform_correction(client, correction, responses_df, unique_id, vector_store, summary_df)
        print("✓ Corrections applied")
    else:
        print("\nStep 3: No corrections needed - second LLM approved all responses!")
    
    return responses_df, correction


### Running the model

In [339]:
## Running this on my remaining groundtruth names, with more descriptive error handling

# Initialize all_responses as an empty DataFrame before starting the loop
all_responses = pd.DataFrame()

for unique_id in groundtruth_list:
    print(f"Processing {unique_id}")
    try:
        responses_df, corrections = full_evaluation_with_correction(client, unique_id, vector_store, groundtruth_summaries)
        
        # Ensure 'unique_id' is a column in responses_df, not just an index
        if 'unique_id' not in responses_df.columns:
            if responses_df.index.name == 'unique_id':
                responses_df = responses_df.reset_index()
            else:
                responses_df['unique_id'] = unique_id

        all_responses = pd.concat([all_responses, responses_df], ignore_index=True)
    except Exception as e:
        import traceback
        print(f"\n{'='*60}")
        print(f"Error processing document: {unique_id}")
        print(f"Exception type: {type(e).__name__}")
        print(f"Exception message: {e}")
        print("Full traceback below:")
        traceback.print_exc()
        print(f"{'='*60}\n")
        # Optionally, log or save the failed ID for later review
        # with open("failed_groundtruth_ids.txt", "a") as f:
        #     f.write(f"{unique_id}: {type(e).__name__} - {e}\n")

Processing 3Oahxp3pihpVDQVt

Evaluating and correcting: 3Oahxp3pihpVDQVt

Step 1: First LLM generating initial responses...
Question 2 response:
{
  "categories": [
    "Domestic real economy decarbonization measures with a financial component",
    "Domestic real economy adaptation, compensation and resilience measures with a financial component"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Products", "subject": "Debt instruments"},
    {"category": "Entities", "subject": "Non-financial corporations (excluding state-owned enterprises)"},
    {"category": "Entities", "subject": "Non-financial state-owned enterprises"},
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Externalities",
    "Public goods",
    "Information asymmetry",
    "Coordination"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {"category": "Market-based", "policy": "Concessio

In [340]:
all_responses

,question_2,question_3,question_4,question_5,question_6,unique_id
0,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Debt instruments""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations (excluding state-owned enterprises)""},\n {""category"": ""Entities"", ""subject"": ""Non-financial state-owned enterprises""},\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""}\n ]\n}","{\n ""subjects"": [\n ""Externalities"",\n ""Public goods"",\n ""Information asymmetry""\n ]\n}","{\n ""policy_layer"": [\n {""category"": ""Command-and-control"", ""policy"": ""Investment mandates or constraints""}\n ],\n ""implementation_layer"": [\n {""category"": ""Market-based"", ""policy"": ""Concessional finance, subsidies and grants""},\n {""category"": ""Informational"", ""policy"": ""Labelling and standards""},\n {""category"": ""Informational"", ""policy"": ""Auditing, substantiation and verification requirements""},\n {""category"": ""Informational"", ""policy"": ""Mandatory information disclosure (product- or entity-level)""}\n ]\n}","{\n ""announcement_date"": ""November 2023"",\n ""entry_into_force_date"": """",\n ""end_date"": """",\n ""adopting_authority_name"": ""Government of Canada, Department of Finance Canada"",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}",3Oahxp3pihpVDQVt
1,"{\n ""categories"": [\n ""Domestic real economy adaptation, compensation and resilience measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Grants""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations""},\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""}\n ]\n}","{\n ""subjects"": [\n ""Public goods"",\n ""Externalities""\n ]\n}","{\n ""categories_and_policies"": [\n {""category"": ""Command-and-control"", ""policy"": ""Investment mandates or constraints""},\n {""category"": ""Market-based"", ""policy"": ""Concessional finance, subsidies and grants""}\n ]\n}","```json\n{\n ""announcement_date"": ""2018"",\n ""entry_into_force_date"": ""2018"",\n ""end_date"": ""2033"",\n ""adopting_authority_name"": ""Housing, Infrastructure and Communities Canada"",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}\n```",tnaWzSNVgSLTv7XX
2,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Entities"", ""subject"": ""Sovereign funds and investment-focused state-owned entities""},\n {""category"": ""Products"", ""subject"": ""Derivatives""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations""}\n ]\n}","{\n ""subjects"": [\n ""Externalities"",\n ""Public goods"",\n ""Information asymmetry""\n ]\n}","{\n ""policy_layer"": {\n ""category"": ""Command-and-control"",\n ""policy"": ""Investment mandates or constraints""\n },\n ""implementation_layer"": [\n {\n ""category"": ""Market-based"",\n ""policy"": ""Concessional finance, subsidies and grants""\n }\n ]\n}","```json\n{\n ""announcement_date"": ""2023-10-25"",\n ""entry_into_force_date"": ""2022-12"",\n ""end_date"": """",\n ""adopting_authority_name"": ""Canada Development Investment Corporation (CDEV)"",\n ""adopting_authority_type"": ""Government entity"",\n ""policy_geographical_focus"": ""Domestic""\n}\n```",K96A0SZrnego6D4a
3,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Other forms of direct payments""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations""},\n {""category""

In [341]:
# Let's merge all_responses with groundtruth_df on unique_id and rename columns accordingly

# Ensure both DataFrames have a unique_id column (reset index if necessary)
all_responses = all_responses.reset_index() if "unique_id" not in all_responses.columns else all_responses.copy()
groundtruth_df = groundtruth_df.reset_index() if "unique_id" not in groundtruth_df.columns else groundtruth_df.copy()

# Rename columns for clarity
all_responses_renamed = all_responses.rename(columns={
    "question_2": "response_2",
    "question_3": "response_3",
    "question_4": "response_4",
    "question_5": "response_5",
    "question_6": "response_6",
})
groundtruth_df_renamed = groundtruth_df.rename(columns={
    "question_2": "groundtruth_2",
    "question_3": "groundtruth_3",
    "question_4": "groundtruth_4",
    "question_5": "groundtruth_5",
    "question_6": "groundtruth_6",
})

# Subset just to those columns plus unique_id
response_cols = ["unique_id"] + [col for col in all_responses_renamed.columns if col.startswith("response_")]
gt_cols = ["unique_id"] + [col for col in groundtruth_df_renamed.columns if col.startswith("groundtruth_")]
all_responses_sub = all_responses_renamed[response_cols]
groundtruth_sub = groundtruth_df_renamed[gt_cols]

# Merge on unique_id, keeping all unique_ids in either DataFrame (outer join)
merged = pd.merge(all_responses_sub, groundtruth_sub, on="unique_id", how="outer")

# For readability, interleave the question columns for comparison
cols_order = ["unique_id"]
for i in range(2, 7):
    if f"response_{i}" in merged.columns:
        cols_order.append(f"response_{i}")
    if f"groundtruth_{i}" in merged.columns:
        cols_order.append(f"groundtruth_{i}")

merged = merged[cols_order]

merged


,unique_id,response_2,groundtruth_2,response_3,groundtruth_3,response_4,groundtruth_4,response_5,groundtruth_5,response_6
0,2qn97s0DnXYe2qH8,"{\n ""categories"": [\n ""Domestic financial sector policy""\n ]\n}",Domestic financial sector policy,"{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Investment funds (as products)""},\n {""category"": ""Entities"", ""subject"": ""Private investment fund managers""},\n {""category"": ""Entities"", ""subject"": ""Retail consumers""}\n ]\n}",Product: investment funds; Entity: private investment fund managers,"{\n ""subjects"": [\n ""Information asymmetry""\n ]\n}",Coordination and information asymmetry,"{\n ""categories_and_policies"": [\n {""category"": ""Informational"", ""policy"": ""Mandatory information disclosure (product- or entity-level)""},\n {""category"": ""Informational"", ""policy"": ""Deceptive marketing rules and guidelines""}\n ]\n}",Informational: labeling and standards; mandatory information disclosure,"{\n ""announcement_date"": ""2022-01-19"",\n ""entry_into_force_date"": ""2024-03-07"",\n ""end_date"": """",\n ""adopting_authority_name"": ""Canadian Securities Administrators (CSA)"",\n ""adopting_authority_type"": ""Regulatory agency"",\n ""policy_geographical_focus"": ""Domestic""\n}"
1,3Oahxp3pihpVDQVt,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}",Domestic real economy decarbonization measures with a financial component; Domestic real economy adaptation and compensation measures with a financial component; Domestic financial sector policy,"{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Debt instruments""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations (excluding state-owned enterprises)""},\n {""category"": ""Entities"", ""subject"": ""Non-financial state-owned enterprises""},\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""}\n ]\n}",Product: Debt instruments,"{\n ""subjects"": [\n ""Externalities"",\n ""Public goods"",\n ""Information asymmetry""\n ]\n}",Coordination; information asymmetry,"{\n ""policy_layer"": [\n {""category"": ""Command-and-control"", ""policy"": ""Investment mandates or constraints""}\n ],\n ""implementation_layer"": [\n {""category"": ""Market-based"", ""policy"": ""Concessional finance, subsidies and grants""},\n {""category"": ""Informational"", ""policy"": ""Labelling and standards""},\n {""category"": ""Informational"", ""policy"": ""Auditing, substantiation and verification requirements""},\n {""category"": ""Informational"", ""policy"": ""Mandatory information disclosure (product- or entity-level)""}\n ]\n}",Market-based: market infrastructure; Informational: Labelling and standards,"{\n ""announcement_date"": ""November 2023"",\n ""entry_into_force_date"": """",\n ""end_date"": """",\n ""adopting_authority_name"": ""Government of Canada, Department of Finance Canada"",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}"
2,DAum9XBl9BxXQdTP,"{\n ""categories"": [\n ""Domestic financial sector policy""\n ]\n}",Domestic financial sector policy,"{\n ""categories_and_subjects"": [\n {""category"": ""Entities"", ""subject"": ""Commercial banks, cooperatives, credit unions and other deposit-taking institutions""},\n {""category"": ""Entities"", ""subject"": ""Insurers and reinsurers (not as investors)""}\n ]\n}","Entity: Commercial banks, cooperatives, credit unions and other deposit-taking institution; Insurers","{\n ""subjects"": [\n ""Information asymmetry"",\n ""Externalities"",\n ""Public goods""\n ]\n}",Information asymmetry; myopia; systemic risk; coordination,"{\n ""categories_and_policies"": [\n {""category"": ""Command-and-control"", ""policy"": ""Risk identification and management""},\n {""category"": ""Informational"", ""policy"": ""Training, literacy and awareness measures""}\n ]\n}

In [342]:
id_to_name_map = pd.read_csv('../data/unique_id_to_name.csv')

In [343]:
# Add the 'name' column to merged (formerly all_results) by mapping unique_id to name using id_to_name_map
# id_to_name_map is assumed to have columns: 'unique_id' and 'name'
id_to_name_map_dict = dict(zip(id_to_name_map['unique_id'], id_to_name_map['name']))
merged.insert(1, 'name', merged['unique_id'].map(id_to_name_map_dict))


In [344]:
import os

# Ensure the results directory exists
os.makedirs('../results', exist_ok=True)

# Save merged DataFrame to the results folder as a CSV
merged.to_csv('../results/large_model_with_review_run_2.csv', index=False)

## Evaluation Framework

In [346]:
# Prepare evaluation data
# Ensure we have both responses and ground truth aligned

# Check if merged dataframe exists (from cell 47), otherwise recreate it
if 'merged' not in globals():
    print("Recreating merged dataframe...")
    # Ensure both DataFrames have a unique_id column (reset index if necessary)
    all_responses_prep = all_responses.reset_index() if "unique_id" not in all_responses.columns else all_responses.copy()
    groundtruth_df_prep = groundtruth_df.reset_index() if "unique_id" not in groundtruth_df.columns else groundtruth_df.copy()
    
    # Rename columns for clarity
    all_responses_renamed = all_responses_prep.rename(columns={
        "question_2": "response_2",
        "question_3": "response_3",
        "question_4": "response_4",
        "question_5": "response_5",
        "question_6": "response_6",
    })
    groundtruth_df_renamed = groundtruth_df_prep.rename(columns={
        "question_2": "groundtruth_2",
        "question_3": "groundtruth_3",
        "question_4": "groundtruth_4",
        "question_5": "groundtruth_5",
        "question_6": "groundtruth_6",
    })
    
    # Subset just to those columns plus unique_id
    response_cols = ["unique_id"] + [col for col in all_responses_renamed.columns if col.startswith("response_")]
    gt_cols = ["unique_id"] + [col for col in groundtruth_df_renamed.columns if col.startswith("groundtruth_")]
    all_responses_sub = all_responses_renamed[response_cols]
    groundtruth_sub = groundtruth_df_renamed[gt_cols]
    
    # Merge on unique_id
    merged = pd.merge(all_responses_sub, groundtruth_sub, on="unique_id", how="inner")

# Get the merged dataframe
eval_data = merged.copy()

# Filter to only rows where we have both response and groundtruth for at least one question
# (We'll evaluate each question separately)
eval_data = eval_data[eval_data['unique_id'].notna()]

print(f"Evaluation dataset prepared: {len(eval_data)} documents")
print(f"\nDocuments in evaluation set:")
for uid in eval_data['unique_id'].tolist():
    print(f"  - {uid}")


Evaluation dataset prepared: 14 documents

Documents in evaluation set:
  - 2qn97s0DnXYe2qH8
  - 3Oahxp3pihpVDQVt
  - DAum9XBl9BxXQdTP
  - G6RmGheTDI0EhzXd
  - K96A0SZrnego6D4a
  - OzXmCrgCYiGc1Nk2
  - QHLNGcTNy1519Q5Y
  - cbRMF5EZrlSlsqq1
  - eRFNvUxTiyJ5mgky
  - lf0vaU6VpW26FhQV
  - rxkDG0Ai5LbWIkAw
  - tnaWzSNVgSLTv7XX
  - vAu7yXhg2dOCsQzf
  - zm6ny9KbxXZrYyEM


In [347]:
# Create a custom evaluator function for classification tasks
# This will use a rubric-based approach with a judge model that assesses
# whether responses are reasonable and aligned with the ground truth thinking

def create_classification_evaluator(question_num, question_description):
    """
    Create an evaluator that compares model responses to ground truth for classification tasks.
    Uses a nuanced rubric-based approach where a judge model evaluates if the response is 
    reasonable and in line with the thinking of the ground truth evaluator.
    """
    
    def evaluate_classification(sample, response):
        """
        Evaluate a single classification response against ground truth.
        
        Args:
            sample: Dict with 'groundtruth' key containing the expected answer
            response: The model's response to evaluate
            
        Returns:
            Dict with 'score' (0.0 to 1.0) and 'reasoning'
        """
        groundtruth = sample.get('groundtruth', '').strip()
        model_response = str(response).strip() if response else ''
        
        # For exact match (can be enhanced with fuzzy matching)
        # Normalize both responses for comparison
        groundtruth_normalized = groundtruth.lower().strip()
        model_response_normalized = model_response.lower().strip()
        
        # Check for exact match
        if groundtruth_normalized == model_response_normalized:
            return {
                'score': 1.0,
                'reasoning': 'Exact match between model response and ground truth'
            }
        
        # Use a judge model to evaluate with nuanced scoring
        # Create a prompt for the judge that focuses on reasonableness, alignment, and strict adherence to requirements
        judge_prompt = f"""You are evaluating a classification task for climate finance policy instruments.

Question: {question_description}

Ground Truth Answer: {groundtruth}

Model Response: {model_response}

Your task is to assess whether the model response is reasonable and in line with the thinking of the ground truth evaluator. In general, be lenient if the reasoning is sound as this is all ambiguous. 
This is not just about exact matching - consider:

1. **Reasonableness**: Is the model's response a reasonable interpretation of the policy instrument given the classification criteria?
2. **Alignment with thinking**: Does the model's response demonstrate similar reasoning and categorization logic as the ground truth?
3. **Correctness**: Does the response capture the same essential classification/categorization as the ground truth?

CRITICAL EVALUATION CRITERIA:

**Strict Adherence to Options**:
- The model response should only use categories, entities, market failures, and policy instruments that are in the classification schema
- If the response includes categories NOT in the provided list (e.g., "indigenous organizations", "non-profit organizations" when not listed), this is a significant issue
- However, note that "Public bodies" includes Indigenous organizations and governments

**Consistency of Terminology**:
- For Question 2, responses should use the EXACT category names from the schema consistently
- Do not accept responses that mix different phrasings for the same concept (e.g., alternating between "real economy mitigation" and "real economy decarbonization measures with a financial component")
- The ground truth uses specific terminology - the model should match that terminology, not use alternative phrasings

**Completeness**:
- For Question 5, responses must include BOTH the category AND the policy instrument (e.g., "Market-based: Concessional finance, subsidies and grants")
- If a response has the category but not the policy instrument, or vice versa, this is incomplete

**Question-Specific Considerations**:
- Question 2: Ensure consistent use of exact category names. 
- Question 3: Verify all entities are from the provided list. The model should not add entities not in the schema
- Question 4: Consider that there are correlations - MIT measures often relate to externalities/public goods and concessional finance/subsidies/taxes. Market failures can be ambiguous, so be lenient if the reasoning is sound
- Question 5: Must include both category and policy instrument. Note that public investment in infrastructure counts as "Concessional finance, subsidies and grants" NOT "Market infrastructure"

Consider that:
- The response may use different wording but convey the same meaning (but prefer exact terminology matches)
- The response may be partially correct (e.g., identifies some but not all categories)
- Minor formatting differences are acceptable
- The core meaning, categories, and reasoning should align
- However, terminology consistency and strict adherence to provided options are important

Provide a nuanced score from 0.0 to 1.0 where:
- 1.0 = Perfect match or equivalent reasoning with correct terminology and complete response
- 0.8-0.9 = Very close, minor differences in wording or one missing element, but uses correct terminology
- 0.6-0.7 = Reasonable response that aligns with ground truth thinking but has some differences (terminology issues, missing elements, or going beyond options)
- 0.4-0.5 = Partially correct, shows some understanding but misses key elements or uses incorrect terminology
- 0.2-0.3 = Somewhat reasonable but significant differences from ground truth (wrong categories, incomplete, inconsistent)
- 0.0-0.1 = Incorrect or demonstrates different reasoning entirely

Format your response as JSON:
{{"score": <float between 0.0 and 1.0>, "reasoning": "<detailed explanation addressing: (1) terminology consistency, (2) adherence to provided options, (3) completeness, (4) alignment with ground truth>"}}
"""
        
        try:
            judge_response = client.chat.completions.create(
                model=MODEL,
                temperature=TEMPERATURE,
                messages=[
                    {"role": "system", "content": "You are an expert evaluator for classification tasks. You assess whether responses are reasonable and demonstrate sound reasoning, not just exact matches."},
                    {"role": "user", "content": judge_prompt}
                ],
                response_format={"type": "json_object"}
            )
            
            import json
            result = json.loads(judge_response.choices[0].message.content)
            score = float(result.get('score', 0.0))
            # Ensure score is in valid range
            score = max(0.0, min(1.0, score))
            return {
                'score': score,
                'reasoning': result.get('reasoning', 'No reasoning provided')
            }
        except Exception as e:
            print(f"Error in judge evaluation: {e}")
            # Fallback to partial string matching
            if groundtruth_normalized in model_response_normalized or model_response_normalized in groundtruth_normalized:
                return {
                    'score': 0.5,
                    'reasoning': 'Partial match detected (fallback method)'
                }
            return {
                'score': 0.0,
                'reasoning': f'No match found. Error: {str(e)}'
            }
    
    return evaluate_classification

print("Evaluator functions created")


Evaluator functions created


In [348]:
# Run evaluations for each question (excluding question 6 - no ground truth available)
import json
from collections import defaultdict

evaluation_results = {}

# Question descriptions for context
question_descriptions = {
    2: "Categorize whether the instrument primarily targets the financial sector or the real economy",
    3: "Categorize the policy instrument's subject of intervention",
    4: "Categorize the market failure addressed",
    5: "Categorize the type of incentive established"
}

for question_num in range(2, 6):  # Only evaluate questions 2-5 (no ground truth for question 6)
    print(f"\n{'='*60}")
    print(f"Evaluating Question {question_num}")
    print(f"{'='*60}")
    
    response_col = f'response_{question_num}'
    groundtruth_col = f'groundtruth_{question_num}'
    
    # Create evaluator for this question
    evaluator = create_classification_evaluator(
        question_num, 
        question_descriptions[question_num]
    )
    
    # Evaluate each document
    question_results = []
    correct_count = 0
    total_count = 0
    
    for idx, row in eval_data.iterrows():
        unique_id = row['unique_id']
        groundtruth = row[groundtruth_col]
        model_response = row[response_col]
        
        if pd.isna(groundtruth) or pd.isna(model_response):
            continue
        
        sample = {'groundtruth': str(groundtruth)}
        result = evaluator(sample, model_response)
        
        score = result['score']
        reasoning = result.get('reasoning', '')
        
        question_results.append({
            'unique_id': unique_id,
            'groundtruth': str(groundtruth),
            'model_response': str(model_response),
            'score': score,
            'reasoning': reasoning
        })
        
        # Use a threshold of 0.7 for "correct" (reasonable and well-aligned)
        # But track the nuanced scores for analysis
        if score >= 0.5:
            correct_count += 1
        total_count += 1
        
        # Display with more nuanced indicators
        if score >= 0.9:
            indicator = '✓✓'  # Excellent
        elif score >= 0.7:
            indicator = '✓'   # Good
        elif score >= 0.5:
            indicator = '~'    # Partial
        else:
            indicator = '✗'   # Poor
        
        print(f"  {unique_id}: {indicator} (score: {score:.2f})")
    
    # Calculate metrics
    accuracy = correct_count / total_count if total_count > 0 else 0.0
    avg_score = sum(r['score'] for r in question_results) / len(question_results) if question_results else 0.0
    
    evaluation_results[f'question_{question_num}'] = {
        'results': question_results,
        'accuracy': accuracy,
        'average_score': avg_score,
        'total_evaluated': total_count,
        'correct': correct_count
    }
    
    print(f"\nQuestion {question_num} Summary:")
    print(f"  Accuracy: {accuracy:.2%} ({correct_count}/{total_count})")
    print(f"  Average Score: {avg_score:.3f}")

print(f"\n{'='*60}")
print("All evaluations complete!")
print(f"{'='*60}")



Evaluating Question 2


  2qn97s0DnXYe2qH8: ✓✓ (score: 1.00)
  3Oahxp3pihpVDQVt: ✓✓ (score: 0.90)
  DAum9XBl9BxXQdTP: ✓✓ (score: 1.00)
  G6RmGheTDI0EhzXd: ✓✓ (score: 1.00)
  K96A0SZrnego6D4a: ✓✓ (score: 1.00)
  OzXmCrgCYiGc1Nk2: ✓ (score: 0.70)
  QHLNGcTNy1519Q5Y: ✓ (score: 0.70)
  cbRMF5EZrlSlsqq1: ✓✓ (score: 1.00)
  eRFNvUxTiyJ5mgky: ✓✓ (score: 1.00)
  lf0vaU6VpW26FhQV: ✓✓ (score: 0.90)
  rxkDG0Ai5LbWIkAw: ✓✓ (score: 1.00)
  tnaWzSNVgSLTv7XX: ✓✓ (score: 0.90)
  vAu7yXhg2dOCsQzf: ✓✓ (score: 1.00)
  zm6ny9KbxXZrYyEM: ✓✓ (score: 1.00)

Question 2 Summary:
  Accuracy: 100.00% (14/14)
  Average Score: 0.936

Evaluating Question 3
  2qn97s0DnXYe2qH8: ✓✓ (score: 0.90)
  3Oahxp3pihpVDQVt: ✓✓ (score: 0.90)
  DAum9XBl9BxXQdTP: ✓✓ (score: 0.95)
  G6RmGheTDI0EhzXd: ✓ (score: 0.70)
  K96A0SZrnego6D4a: ✓ (score: 0.70)
  OzXmCrgCYiGc1Nk2: ~ (score: 0.60)
  QHLNGcTNy1519Q5Y: ~ (score: 0.60)
  cbRMF5EZrlSlsqq1: ✓✓ (score: 0.90)
  eRFNvUxTiyJ5mgky: ~ (score: 0.60)
  lf0vaU6VpW26FhQV: ✗ (score: 0.30)
  rxkDG0Ai5LbWIkAw: ✗ (sc

In [349]:
# Display comprehensive evaluation summary
import pandas as pd

summary_data = []
for q_num in range(2, 6):  # Only questions 2-5 (no ground truth for question 6)
    q_key = f'question_{q_num}'
    if q_key in evaluation_results:
        results = evaluation_results[q_key]
        summary_data.append({
            'Question': f'Question {q_num}',
            'Description': question_descriptions[q_num],
            'Accuracy': f"{results['accuracy']:.2%}",
            'Average Score': f"{results['average_score']:.3f}",
            'Correct': results['correct'],
            'Total': results['total_evaluated']
        })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("EVALUATION SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)



EVALUATION SUMMARY
  Question                                                                                  Description Accuracy Average Score  Correct  Total
Question 2 Categorize whether the instrument primarily targets the financial sector or the real economy  100.00%         0.936       14     14
Question 3                                   Categorize the policy instrument's subject of intervention   85.71%         0.729       12     14
Question 4                                                      Categorize the market failure addressed  100.00%         0.786       14     14
Question 5                                                 Categorize the type of incentive established   92.86%         0.771       13     14


In [350]:
# Detailed results for each question - view lower-scoring predictions
# Shows predictions with scores below 0.7 (threshold for "good" alignment)
for q_num in range(2, 6):  # Only questions 2-5 (no ground truth for question 6)
    q_key = f'question_{q_num}'
    if q_key not in evaluation_results:
        continue
    
    results = evaluation_results[q_key]['results']
    incorrect = [r for r in results if r['score'] < 0.7]  # Show scores below "good" threshold
    
    if incorrect:
        print(f"\n{'='*80}")
        print(f"Question {q_num} - Lower-Scoring Predictions (score < 0.7) ({len(incorrect)}/{len(results)})")
        print(f"{'='*80}")
        
        for item in incorrect:
            print(f"\nDocument: {item['unique_id']}")
            print(f"Ground Truth: {item['groundtruth']}")
            print(f"Model Response: {item['model_response']}")
            print(f"Score: {item['score']:.2f}")
            print(f"Reasoning: {item['reasoning']}")
            print("-" * 80)
    else:
        print(f"\nQuestion {q_num}: All predictions correct! ✓")



Question 2: All predictions correct! ✓

Question 3 - Lower-Scoring Predictions (score < 0.7) (5/14)

Document: OzXmCrgCYiGc1Nk2
Ground Truth: Products: equity instruments; debt instruments; Entity: [all]
Model Response: {
  "categories_and_subjects": [
    {"category": "Entities", "subject": "Private institutional investors"},
    {"category": "Entities", "subject": "Private investment fund managers"},
    {"category": "Entities", "subject": "Private sector asset managers"},
    {"category": "Entities", "subject": "Non-financial corporations (excluding state-owned enterprises)"}
  ]
}
Score: 0.60
Reasoning: (1) Terminology Consistency: The model uses the category 'Entities', which is consistent with the schema. The subjects listed (e.g., 'Private institutional investors', 'Private investment fund managers') are phrased in a way that aligns with the schema's terminology, though not necessarily exact matches. (2) Adherence to Provided Options: The model adheres to the schema by only lis

In [351]:
# Save evaluation results to a file for future reference
import json
from datetime import datetime

eval_output = {
    'timestamp': datetime.now().isoformat(),
    'total_documents': len(eval_data),
    'evaluation_results': {}
}

for q_key, q_results in evaluation_results.items():
    eval_output['evaluation_results'][q_key] = {
        'accuracy': q_results['accuracy'],
        'average_score': q_results['average_score'],
        'total_evaluated': q_results['total_evaluated'],
        'correct': q_results['correct'],
        'detailed_results': q_results['results']
    }

output_path = '../data/evaluation_results_large_with_review_run_2.json'
with open(output_path, 'w') as f:
    json.dump(eval_output, f, indent=2)

print(f"Evaluation results saved to {output_path}")


Evaluation results saved to ../data/evaluation_results_large_with_review_run_2.json


____

## Assessing Results

In [352]:
import glob
import json
import pandas as pd

# Read in all files that start with ../data/evaluation_results
eval_files = [f for f in glob.glob('../data/evaluation_results*')]
dataframes = {}

for fname in eval_files:
    try:
        with open(fname, 'r') as f:
            data = json.load(f)
            # Flatten nested results if possible; focus on 'evaluation_results'
            if 'evaluation_results' in data:
                records = []
                for k, v in data['evaluation_results'].items():
                    if 'detailed_results' in v:
                        for row in v['detailed_results']:
                            row_flat = row.copy()
                            row_flat['question_key'] = k
                            row_flat['accuracy'] = v.get('accuracy')
                            row_flat['average_score'] = v.get('average_score')
                            row_flat['total_evaluated'] = v.get('total_evaluated')
                            row_flat['correct'] = v.get('correct')
                            records.append(row_flat)
                    else:
                        row_flat = v.copy()
                        row_flat['question_key'] = k
                        records.append(row_flat)
                df = pd.DataFrame(records)
            else:
                # Save entire data as single-row dataframe if structure unknown
                df = pd.DataFrame([data])
            
            # Unique variable name, e.g., df_evaluation_results, df_evaluation_results_large, etc.
            key = fname.split('/')[-1].replace('.json', '').replace('.', '_')
            var_name = f"df_{key}"
            dataframes[var_name] = df
            globals()[var_name] = df  # Assign to local/global namespace

    except Exception as e:
        print(f"Error reading {fname}: {e}")

print(f"Read {len(dataframes)} evaluation result files and saved as DataFrames with unique names:")
for var_name, df in dataframes.items():
    print(f"- {var_name} (rows: {len(df)}, columns: {list(df.columns)})")


Read 8 evaluation result files and saved as DataFrames with unique names:
- df_evaluation_results_large_no_review (rows: 56, columns: ['unique_id', 'groundtruth', 'model_response', 'score', 'reasoning', 'question_key', 'accuracy', 'average_score', 'total_evaluated', 'correct'])
- df_evaluation_results_small_with_review (rows: 56, columns: ['unique_id', 'groundtruth', 'model_response', 'score', 'reasoning', 'question_key', 'accuracy', 'average_score', 'total_evaluated', 'correct'])
- df_evaluation_results_large_with_review_run_2 (rows: 56, columns: ['unique_id', 'groundtruth', 'model_response', 'score', 'reasoning', 'question_key', 'accuracy', 'average_score', 'total_evaluated', 'correct'])
- df_evaluation_results_large_with_review (rows: 56, columns: ['unique_id', 'groundtruth', 'model_response', 'score', 'reasoning', 'question_key', 'accuracy', 'average_score', 'total_evaluated', 'correct'])
- df_evaluation_results_small_no_review_run_2 (rows: 56, columns: ['unique_id', 'groundtruth',

In [353]:
def summarize_scores(df, title=None):
    """
    Groups the given DataFrame by 'question_key', counts passing answers, and computes average scores.
    Optionally prints a title above the summary.

    Args:
        df (pd.DataFrame): DataFrame with at least columns ['question_key', 'average_score']
        title (str, optional): Title to print above the summary. Default is None.

    Returns:
        pd.DataFrame: Grouped and aggregated DataFrame
    """
    grouped = df.groupby('question_key').agg(
        passing_answers=('score', lambda x: f"{(x > 0.5).sum()} / 14"),
        avg_score=('average_score', 'mean')
    ).reset_index()
    if title:
        print(title)
    print(grouped)
    return grouped

In [354]:
summarize_scores(df_evaluation_results_large_no_review_run_2, 'large model no review')

large model no review
  question_key passing_answers  avg_score
0   question_2         14 / 14   0.957143
1   question_3         13 / 14   0.817857
2   question_4         14 / 14   0.867857
3   question_5         13 / 14   0.807143


,question_key,passing_answers,avg_score
0,question_2,14 / 14,0.957143
1,question_3,13 / 14,0.817857
2,question_4,14 / 14,0.867857
3,question_5,13 / 14,0.807143


In [355]:
summarize_scores(df_evaluation_results_large_with_review_run_2, 'large model with review')

large model with review
  question_key passing_answers  avg_score
0   question_2         14 / 14   0.935714
1   question_3         12 / 14   0.728571
2   question_4         14 / 14   0.785714
3   question_5         11 / 14   0.771429


,question_key,passing_answers,avg_score
0,question_2,14 / 14,0.935714
1,question_3,12 / 14,0.728571
2,question_4,14 / 14,0.785714
3,question_5,11 / 14,0.771429


In [356]:
summarize_scores(df_evaluation_results_small_with_review_run_2, 'small model with review')

small model with review
  question_key passing_answers  avg_score
0   question_2         12 / 14   0.800000
1   question_3         11 / 14   0.678571
2   question_4         14 / 14   0.700000
3   question_5         11 / 14   0.692857


,question_key,passing_answers,avg_score
0,question_2,12 / 14,0.800000
1,question_3,11 / 14,0.678571
2,question_4,14 / 14,0.700000
3,question_5,11 / 14,0.692857


In [357]:
summarize_scores(df_evaluation_results_small_no_review_run_2, 'small model no review')

small model no review
  question_key passing_answers  avg_score
0   question_2         12 / 14   0.771429
1   question_3         12 / 14   0.685714
2   question_4         14 / 14   0.785714
3   question_5         11 / 14   0.735714


,question_key,passing_answers,avg_score
0,question_2,12 / 14,0.771429
1,question_3,12 / 14,0.685714
2,question_4,14 / 14,0.785714
3,question_5,11 / 14,0.735714


In [358]:
# Show the full reasoning without truncation (disable ellipsis)
pd.set_option('display.max_colwidth', None)
subset = df_evaluation_results_large_with_review[df_evaluation_results_large_with_review['question_key'] == 'question_3']

from IPython.display import display_html

def side_by_side_print(df, cols=['score', 'reasoning']):
    html = ""
    for idx, row in df[cols].iterrows():
        html += f"<div style='display: flex; margin-bottom: 2em;'>"
        html += f"<div style='flex:1; margin-right:2em;'><strong>Score:</strong> {row['score']}</div>"
        html += f"<div style='flex:6;'><strong>Reasoning:</strong><br>{row['reasoning']}</div>"
        html += "</div><hr>"
    display_html(html, raw=True)

side_by_side_print(subset)

Score: 0.6 Reasoning: The model response identifies 'Products: Investment funds (as products)', which partially aligns with the ground truth answer of 'Product: investment funds; Entity: private investment fund managers'. 

(1) Terminology consistency: The model uses 'Products' instead of the singular 'Product', which is a minor inconsistency but acceptable. However, it omits the 'Entity' component entirely, which is a significant omission.

(2) Adherence to provided options: 'Product: investment funds' is a valid category from the schema, so the model adheres to the classification schema in that respect. However, by not including the entity, it fails to fully adhere to the expected format.

(3) Completeness: The response is incomplete because it only includes the product and not the entity. The question explicitly asks for the 'subject of intervention', which includes both product and entity.

(4) Alignment with ground truth: The model demonstrates partial alignment with the ground truth by correctly identifying the product, but it does not show full reasoning or categorization logic by omitting the entity.

Overall, the response shows some understanding and partial correctness but lacks completeness and full alignment with the ground truth. Score: 1.0 Reasoning: The model response 'Products: Debt instruments' is a perfect match to the ground truth 'Product: Debt instruments' with only a minor pluralization difference ('Products' vs. 'Product'). This does not affect the meaning or classification. (1) Terminology consistency is maintained, as 'Debt instruments' is used exactly as in the schema. (2) The response adheres strictly to the provided classification options. (3) The response is complete, including both the category ('Product') and the specific subject ('Debt instruments'). (4) The model demonstrates the same reasoning and categorization logic as the ground truth. The pluralization is a negligible formatting variation and does not detract from the correctness or alignment. Score: 0.6 Reasoning: The model response demonstrates a reasonable interpretation of the policy instrument as a supervisory tool focused on risk assessment, which may not directly involve financial products. However, it fails to adhere to the classification schema by not selecting from the provided list of entities. The ground truth correctly identifies the subject of intervention as 'Commercial banks, cooperatives, credit unions and other deposit-taking institution; Insurers', which are valid categories in the schema. The model instead states 'None of the listed financial products apply', which is not a valid response under the schema and confuses the subject of intervention (which should be an entity) with the type of financial product. This indicates a misunderstanding of the classification task. While the reasoning is somewhat aligned in recognizing the supervisory nature of the instrument, the model does not use the required terminology or categories, and the response is incomplete. Therefore, it earns a mid-range score for partial alignment but significant issues in terminology and adherence to schema. Score: 0.3 Reasoning: The model response 'Products: Grants' does not align with the question, which asks for the 'subject of intervention'—i.e., the entity being targeted by the policy instrument. The ground truth correctly identifies this as 'Entity: non-financial firms', which is a valid category from the schema. In contrast, the model response refers to a type of policy instrument ('Grants') and a non-schema category ('Products'), which is not part of the classification schema for subjects of intervention. This indicates a misunderstanding of the question and a failure to use the correct classification category. While 'Grants' may be a relevant policy instrument, it does not answer the question about the subject of intervention. Therefore, the response is significantly misaligned with the ground truth in both terminology and reasoning, though 

_____

## Running the chosen model on the rest of the data

In [ ]:
MODEL = "openai/chatgpt-4o-latest"
TEMPERATURE = 0.0

In [288]:
names_df

,name,First link,Second link,Third link,source,Provincial,Duplicate,Plan or strategy,Climate finance?,Unnamed: 9,Criterion 1,Criterion 2,Criterion 3,Criterion 4,unique_id,first_link_url,second_link_url,third_link_url
0,CO2 emission credit system,Link,NaN,NaN,all_docs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DueoEkCNSrdjAkzn,https://www.canada.ca/content/dam/eccc/documents/pdf/climate-change/ghg-offset/indigenous-toolkit/Planning-a-Project-in-GHG-Offset-System-EN.pdf,NaN,NaN
1,Output-Based Pricing System Regulations,Link,NaN,NaN,"all_docs, CPR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fbjFEgjb9FLcJ2e6,https://laws-lois.justice.gc.ca/eng/regulations/SOR-2019-266/index.html,NaN,NaN
2,Low Carbon Economy Fund - Low Carbon Economy Challenge,Link,NaN,NaN,"all_docs, BNDC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zh5iQXVczyUwinGR,https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/challenge/applicant-guide-2023.html,NaN,NaN
3,Low Carbon Economy Fund - Indigenous Leadership Fund,Link,NaN,NaN,"all_docs, BNDC, groundtruth",NaN,NaN,NaN,NaN,NaN,MIT,Entity: Public bodies; non-financial corporations,"Market-based: concessional finance, subsidies and grants",Public goods; externalities; myopia.,cU3N5G1KqrceLGLz,https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/indigenous-leadership.html,NaN,NaN
4,Low Carbon Economy Fund - Implementation Readiness Fund,Link,NaN,NaN,"all_docs, BNDC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3zDkQRaHMFJvpzUc,https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/implementation-readiness.html,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Climate-related disclosure requirement for federal state-owned entities,Link*,"*Only the bit after ""Strengthening Public Climate-related Disclosures""",NaN,groundtruth,NaN,NaN,NaN,YES,NaN,FSP,Entity: Public bodies (excluding state-owned enterprises); public pension plan managers; non-financial state-owned enterprises; export credit agencies.,Informational: mandatory information disclosure,Coordination; information asymmetry; myopia,QHLNGcTNy1519Q5Y,https://budget.canada.ca/2021/report-rapport/p2-en.html,NaN,NaN
155,Standardized Climate Scenario Exercise,Link,NaN,NaN,groundtruth,NaN,NaN,NaN,YES,NaN,FSP,"Entity: Commercial banks, cooperatives, credit unions and other deposit-taking institution; Insurers","Command-and-control: risk management, behavioural duties and governanceInformational: mandatory information disclosure",Information asymmetry; myopia; systemic risk; coordination,DAum9XBl9BxXQdTP,https://www.osfi-bsif.gc.ca/en/data-forms/reporting-returns/standardized-climate-scenario-exercise,NaN,NaN
156,Guidelines on ESG-related investment fund disclosure,Link,NaN,NaN,groundtruth,NaN,NaN,NaN,YES,NaN,FSP,Product: investment funds; Entity: private investment fund managers,Informational: labeling and standards; mandatory information disclosure,Coordination and information asymmetry,2qn97s0DnXYe2qH8,https://www.osc.ca/sites/default/files/2024-03/20240307_81-334_sn-related-investment-fund-disclosure.pdf,NaN,NaN
157,Sustainable investment guidelines,Link,NaN,NaN,groundtruth,NaN,NaN,NaN,YES,NaN,FSP,Products: equity instruments; debt instruments; Entity: [all],Informational: Labelling and standards,Coordination,OzXmCrgCYiGc1Nk2,https://www.canada.ca/en/department-finance/news/2024/10/government-advances-made-in-canada-sustainable-investment-guidelines-to-accelerate-progress-to-net-zero-emissions-by-2050.html,NaN,NaN


In [289]:
# Initialize all_responses as an empty DataFrame before starting the loop
all_responses = pd.DataFrame()


In [290]:
already_processed_ids = set(all_responses['unique_id']) if not all_responses.empty and 'unique_id' in all_responses.columns else set()

for unique_id in remaining_df['unique_id']:
    if unique_id in already_processed_ids:
        print(f"Skipping {unique_id} (already processed)")
        continue
    print(f"Processing {unique_id}")
    try:
        responses_df, corrections = full_evaluation(client, unique_id, vector_store, remaining_summaries)
        all_responses = pd.concat([all_responses, responses_df])
        already_processed_ids.add(unique_id)
    except Exception as e:
        import traceback
        print(f"\n{'='*60}")
        print(f"Error processing document: {unique_id}")
        print(f"Exception type: {type(e).__name__}")
        print(f"Exception message: {e}")
        print("Full traceback below:")
        traceback.print_exc()
        print(f"{'='*60}\n")
        # Optionally, log or save the failed ID for later review
        # with open("failed_groundtruth_ids.txt", "a") as f:
        #     f.write(f"{unique_id}: {type(e).__name__} - {e}\n")

Processing DueoEkCNSrdjAkzn
Question 2 response:
{
  "categories": [
    "Domestic real economy decarbonization measures with a financial component"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Products", "subject": "Environmental markets"},
    {"category": "Entities", "subject": "Non-financial corporations"},
    {"category": "Entities", "subject": "Non-financial state-owned enterprises"},
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Externalities",
    "Missing markets",
    "Information asymmetry",
    "Coordination"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {"category": "Hybrid", "policy": "Emissions trading systems"},
    {"category": "Informational", "policy": "Auditing, substantiation and verification requirements"}
  ]
}
Question 6 response:
```json
{
  "announcement_date": "",
  "entry_into_force_date": "",
  "end_date": "",


Traceback (most recent call last):
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/2383809270.py", line 9, in <module>
    responses_df, corrections = full_evaluation(client, unique_id, vector_store, remaining_summaries)
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/3781096331.py", line 11, in full_evaluation
    raise KeyError(f"unique_id '{unique_id}' not found in summary_df 'title' column")
KeyError: "unique_id 'uNTlaYHBvxulZQnt' not found in summary_df 'title' column"


Question 2 response:
{
  "categories": [
    "Domestic real economy decarbonization measures with a financial component"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Products", "subject": "Debt instruments"},
    {"category": "Entities", "subject": "Non-financial corporations"},
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Externalities",
    "Public goods",
    "Coordination"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {"category": "Market-based", "policy": "Concessional finance, subsidies and grants"}
  ]
}
Question 6 response:
{
  "announcement_date": "May 2022",
  "entry_into_force_date": "",
  "end_date": "",
  "adopting_authority_name": "Canada Infrastructure Bank (CIB)",
  "adopting_authority_type": "Government",
  "policy_geographical_focus": "Domestic"
}
Processing Ulvgo8aIz4XlTRre
Question 2 response:
{
  "categories": [
    "D

Traceback (most recent call last):
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/2383809270.py", line 9, in <module>
    responses_df, corrections = full_evaluation(client, unique_id, vector_store, remaining_summaries)
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/3781096331.py", line 11, in full_evaluation
    raise KeyError(f"unique_id '{unique_id}' not found in summary_df 'title' column")
KeyError: "unique_id 'wbqBzhF1mFuQRVkh' not found in summary_df 'title' column"


Question 2 response:
{
  "categories": [
    "International climate-focused financial support"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Public goods",
    "Information asymmetry"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {"category": "Market-based", "policy": "Concessional finance, subsidies and grants"},
    {"category": "Informational", "policy": "Training, literacy and awareness measures"}
  ]
}
Question 6 response:
{
  "announcement_date": "",
  "entry_into_force_date": "2020-02-06",
  "end_date": "2024-03-31",
  "adopting_authority_name": "Global Affairs Canada",
  "adopting_authority_type": "Government",
  "policy_geographical_focus": "International"
}
Processing gSPJd9kbjkW9mtL1
Question 2 response:
{
  "categories": [
    "International climate-focused financial support"
  ]
}
Question 3 response:

Traceback (most recent call last):
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/2383809270.py", line 9, in <module>
    responses_df, corrections = full_evaluation(client, unique_id, vector_store, remaining_summaries)
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/3781096331.py", line 11, in full_evaluation
    raise KeyError(f"unique_id '{unique_id}' not found in summary_df 'title' column")
KeyError: "unique_id '0uhjJKIpooLf4Z96' not found in summary_df 'title' column"


Question 2 response:
{
  "categories": [
    "Domestic real economy decarbonization measures with a financial component"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Products", "subject": "Grants"},
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"},
    {"category": "Entities", "subject": "Non-profit organizations, non-governmental organizations, independent research organizations, academic institutions, think tanks, lobby groups"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Externalities",
    "Public goods",
    "Information asymmetry"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {"category": "Market-based", "policy": "Concessional finance, subsidies and grants"},
    {"category": "Informational", "policy": "Training, literacy and awareness measures"}
  ]
}
Question 6 response:
```json
{
  "announcement_date": "",
  "entry_into_force_date": "",
  "end_date": "",
  "adopting_authority_

Traceback (most recent call last):
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/2383809270.py", line 9, in <module>
    responses_df, corrections = full_evaluation(client, unique_id, vector_store, remaining_summaries)
  File "/var/folders/br/4mkn2pts7yg8xb1_dfd6g01c0000gn/T/ipykernel_63810/3781096331.py", line 11, in full_evaluation
    raise KeyError(f"unique_id '{unique_id}' not found in summary_df 'title' column")
KeyError: "unique_id 'Lj5YCPIEl18rPWZi' not found in summary_df 'title' column"


Question 2 response:
{
  "categories": [
    "Domestic real economy decarbonization measures with a financial component"
  ]
}
Question 3 response:
{
  "categories_and_subjects": [
    {"category": "Products", "subject": "Grants"},
    {"category": "Entities", "subject": "Public bodies (excluding state-owned enterprises)"},
    {"category": "Entities", "subject": "Non-profit organizations, non-governmental organizations, independent research organizations, academic institutions, think tanks, lobby groups"}
  ]
}
Question 4 response:
{
  "subjects": [
    "Externalities",
    "Public goods",
    "Information asymmetry",
    "Coordination"
  ]
}
Question 5 response:
{
  "categories_and_policies": [
    {
      "category": "Market-based",
      "policy": "Concessional finance, subsidies and grants"
    }
  ]
}
Question 6 response:
```json
{
  "announcement_date": "",
  "entry_into_force_date": "",
  "end_date": "2024-03-31",
  "adopting_authority_name": "Transport Canada",
  "adopting_aut

In [296]:
all_responses = all_responses.reset_index()  # flattens the index; unique_id becomes a column
all_responses

,unique_id,question_2,question_3,question_4,question_5,question_6
0,DueoEkCNSrdjAkzn,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Environmental markets""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations""},\n {""category"": ""Entities"", ""subject"": ""Non-financial state-owned enterprises""},\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""}\n ]\n}","{\n ""subjects"": [\n ""Externalities"",\n ""Missing markets"",\n ""Information asymmetry"",\n ""Coordination""\n ]\n}","{\n ""categories_and_policies"": [\n {""category"": ""Hybrid"", ""policy"": ""Emissions trading systems""},\n {""category"": ""Informational"", ""policy"": ""Auditing, substantiation and verification requirements""}\n ]\n}","```json\n{\n ""announcement_date"": """",\n ""entry_into_force_date"": """",\n ""end_date"": """",\n ""adopting_authority_name"": """",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}\n```"
1,fbjFEgjb9FLcJ2e6,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations (excluding state-owned enterprises)""},\n {""category"": ""Products"", ""subject"": ""Environmental markets""}\n ]\n}","{\n ""subjects"": [\n ""Externalities""\n ]\n}","{\n ""categories_and_policies"": [\n {""category"": ""Hybrid"", ""policy"": ""Emissions trading systems""},\n {""category"": ""Informational"", ""policy"": ""Mandatory information disclosure (product- or entity-level)""},\n {""category"": ""Informational"", ""policy"": ""Auditing, substantiation and verification requirements""}\n ]\n}","{\n ""announcement_date"": """",\n ""entry_into_force_date"": ""2025-10-14"",\n ""end_date"": """",\n ""adopting_authority_name"": ""Government of Canada"",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}"
2,Zh5iQXVczyUwinGR,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component"",\n ""Domestic real economy adaptation, compensation and resilience measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Products"", ""subject"": ""Grants""},\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations (excluding state-owned enterprises)""}\n ]\n}","{\n ""subjects"": [\n ""Externalities"",\n ""Public goods""\n ]\n}","{\n ""categories_and_policies"": [\n {""category"": ""Market-based"", ""policy"": ""Concessional finance, subsidies and grants""}\n ]\n}","```json\n{\n ""announcement_date"": ""2023-11-07"",\n ""entry_into_force_date"": ""2024-02-08"",\n ""end_date"": ""2028-03-31"",\n ""adopting_authority_name"": ""Environment and Climate Change Canada (ECCC)"",\n ""adopting_authority_type"": ""Government"",\n ""policy_geographical_focus"": ""Domestic""\n}\n```"
3,3zDkQRaHMFJvpzUc,"{\n ""categories"": [\n ""Domestic real economy decarbonization measures with a financial component""\n ]\n}","{\n ""categories_and_subjects"": [\n {""category"": ""Entities"", ""subject"": ""Public bodies (excluding state-owned enterprises)""},\n {""category"": ""Entities"", ""subject"": ""Non-financial corporations (excluding state-owned enterprises)""},\n {""category"": ""Entities"", ""subject"": ""Non-financial state-owned enterprises""},\n {""category"": ""Entities"", ""subject"": ""Indigenous communities and authorities""}\n ]\n}","{\n ""subjects"": [\n ""Externalities"",\n ""Public goods"",\n ""Information asymmetry""\n ]\n}","{\n ""categories_and_policies"": [\n {""category"": ""Market-based"", ""policy"": ""Concessional finance, s

In [301]:
#save all_responses
all_responses.to_csv('../results/all_responses_run_2.csv', index=False)
all_responses
groundtruth_df.to_csv('../results/groundtruth_df_run_2.csv', index=False)